# FASTER R-CNN


In [1]:
# Configuration and dependencies
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import imutils.paths as path
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import os

class DatasetHelper:

    TEST = 'test'
    TRAIN = 'train'
    MOT16_TEST = 'MOT16-03'
    MOT16_TRAIN = 'MOT16-04'
    IMAGE = 'img1'
    BOUNDARY_BOXES = 'det\\det.txt'

    def __init__(self, path: str):
        self.path = path
        self.IMAGE_TEST_PATH = ''
        self.IMAGE_TRAIN_PATH = ''
        self.BOUNDARY_BOXES_TEST_PATH = ''
        self.BOUNDARY_TRAIN_BOXES_PATH = ''

    def configure(self):
        self.IMAGE_TEST_PATH = os.path.join(self.path, self.TEST, self.MOT16_TEST, self.IMAGE)
        self.IMAGE_TRAIN_PATH = os.path.join(self.path, self.TRAIN, self.MOT16_TRAIN, self.IMAGE)
        self.BOUNDARY_BOXES_TEST_PATH = os.path.join(self.path, self.TEST, self.MOT16_TEST, self.BOUNDARY_BOXES)
        self.BOUNDARY_TRAIN_BOXES_PATH = os.path.join(self.path, self.TRAIN, self.MOT16_TRAIN, self.BOUNDARY_BOXES)
        train = self.load_data(self.IMAGE_TRAIN_PATH, self.BOUNDARY_TRAIN_BOXES_PATH)
        test = self.load_data(self.IMAGE_TEST_PATH, self.BOUNDARY_BOXES_TEST_PATH)
        return train, test

    def load_data(self, image_path: str, boxes_path: str):
        boxes = []
        output = dict()
        rows = open(boxes_path).read().strip().split('\n')
        for row in rows:
            row = row.split(',')
            frame = row[0]
            id = row[1]
            startX = row[2]
            startY = row[3]
            endX = startX + row[4]
            endY = startY + row[5]
            boxes.append((frame, id, startY, startY, endX, endY))
        all_images = [os.path.join(image_path, f) for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))]
        print (all_images)
        img_index = 0
        while img_index < len(all_images):
            image = cv2.imread(all_images[img_index])
            (h, w) = image.shape[:2]
            image_boxes = [box for box in boxes if box[0] == img_index + 1]
            # scale the bounding box coordinates relative to the spatial
            map(lambda x: (x[0], x[1], x[2] / w, x[3] / h, x[4] / w, x[5] / h), image_boxes)
            image = load_img(all_images[img_index], target_size=(244, 244))
            image = img_to_array(image)
            output[img_index] = (image, image_boxes)
            img_index += 1
        return output

In [2]:
# model path configuration
MODEL_PATH = "detector.h5"
LB_PATH = "lb.pickle"
PLOTS_PATH = "plots"

#base ML params configuration
INIT_LR = 1e-4
NUM_EPOCHS = 20
BATCH_SIZE = 32


In [3]:
#model creation
model = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(244, 244, 3)))
model.trainable = False
flatten = model.output
flatten = Flatten() (flatten)

58900480/58889256 [==============================] - 3s 0us/step


In [4]:
# constructing new layer head for multi-class object detection
bboxHead = Dense(128, activation="relu") (flatten)
bboxHead = Dense(64, activation="relu") (bboxHead)
bboxHead = Dense(32, activation="relu") (bboxHead)
bboxHead = Dense(4, activation="sigmoid", name="bounding_box") (bboxHead)

softmaxHead = Dense(512, activation="relu") (flatten)
softmaxHead = Dropout(0.5) (softmaxHead)
softmaxHead = Dense(512, activation="relu") (softmaxHead)
softmaxHead = Dropout(0.5) (softmaxHead)
softmaxHead = Dense(1, activation="softmax", name="class_label") (softmaxHead)
model = Model(
	inputs=model.input,
	outputs=(bboxHead, softmaxHead))
losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "mean_squared_error"
}

lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}
opt = Adam(lr=INIT_LR)
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 244, 244, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 244, 244, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 244, 244, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

C:\Users\Stasg\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
#Extract images
helper = DatasetHelper(os.path.join(os.path.abspath(''), os.pardir, "dataset"))
train, test = helper.configure()
trainTargets = {
	"class_label": "human",
	"bounding_box": train[:1]
}
# construct a second dictionary, this one for our target testing
# outputs
testTargets = {
	"class_label": "human",
	"bounding_box": test[:1]
}
H = model.fit(
    train[:0], trainTargets,
    validation_data=(test[:1], testTargets),
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=1
)

print("saving model")
model.save(MODEL_PATH, save_format="h5")


['C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000001.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000002.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000003.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000004.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000005.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000006.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000007.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000008.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000009.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000010.jpg', 'C:\\Robotbull\\ObjectTracking\\models\\..\\dataset\\train\\MOT16-04\\img1\\000011.jpg', 'C:\\Robotbull\\Obje

In [ ]:
#result visualization
# plot the total loss, label loss, and bounding box loss
lossNames = ["loss", "class_label_loss", "bounding_box_loss"]
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
(fig, ax) = plt.subplots(3, 1, figsize=(13, 13))
# loop over the loss names
for (i, l) in enumerate(lossNames):
	# plot the loss for both the training and validation data
	title = "Loss for {}".format(l) if l != "loss" else "Total loss"
	ax[i].set_title(title)
	ax[i].set_xlabel("Epoch #")
	ax[i].set_ylabel("Loss")
	ax[i].plot(N, H.history[l], label=l)
	ax[i].plot(N, H.history["val_" + l], label="val_" + l)
	ax[i].legend()
# save the losses figure and create a new figure for the accuracies
plt.tight_layout()
plotPath = os.path.sep.join([PLOTS_PATH, "losses.png"])
plt.savefig(plotPath)
plt.close()